# PDE Modelling for convenience, do not save

## Objectives 

## Model considerations 
* Species
    1. Cell density (sender and pulse cells)
    1. Nutrient density 
    1. mScarlet
* Reactions 
    1. cell growth and diffusion 
        * Cells diffuse very slowly
        * nutrient-dependent growth (from Liu et al 2011, Science) 
        $$  $$ 
    1. Constitutive fluorescence
        * Basal protein expression 
        * initial protein concentration set to fixed point of max nutrient
    1. Dilution and degradation 
        * Assume that all proteins are degradation tagged
    1. Diffusion 
        * Here, you're going to use convoultion of the diffusion kernel
        * Diffusion in/out of cell is considered faster than spatial diffusion at these scales
    1. Parameters
        * We are also assuming, for the moment, that each time point is 6 minutes. Parameters with time dimensions shown below may use different units than the parameter from the cited paper.
        * dx: Length modification of diffusion terms. In the compartmental model, diffusion is calculated via Ficks' first law, where the flux between two adjacent compartments is equal to the flux multiplied by the area of the interface between the components :  
        $\frac{\mathrm{d} C}{\mathrm{d} t} $ 
        in continuous form gives up 
        $\Delta C = D \frac{A}{V} \frac{\Delta C}{\Delta x} = D \frac{2.25 \cdot 5 \cdot \mathrm{scale}^2 \mathrm{mm}^2}{\mathrm{scale} \cdot 2.25^2 \cdot 5 \mathrm{mm}^3} \frac{\Delta C \cdot \mathrm{scale}}{2.25 \mathrm{mm}} = \frac{D \Delta C \mathrm{scale}^2}{2.25^2 \mathrm{mm}^2}$. the dx parameter below is the symbol $A$ in this equation.
        * Dc : Diffusion rate for cells. $7\frac{mm^2}{min}$
        * rc : Division rate of cells. $\frac{1.14}{min}$
        * Kn : Half-point of nutrient availability. 75
        * Dn : Diffusion rate of nutrient. $28\frac{mm^2}{min}$
        * kn : Consumption rate of nutrient by cells
        * Da : Diffusion rate of nutrient. $28\frac{mm^2}{min}$
        * xs : Expression rate of protein. 


In [1]:
# imports
from __future__ import division, print_function

import itertools as itt

import numpy as np
import pandas as pd
import os
import sys
import string
import selenium
import scipy.integrate as itg
import scipy.optimize as opt
import scipy.interpolate as itp
import scipy.ndimage as ndi
import scipy.sparse as sparse

import matplotlib as mpl
mpl.use("Agg")

import itertools

import matplotlib.pyplot as plt 
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg' # Add the path of ffmpeg here!!

import matplotlib.animation as anm
import skimage.measure
import skimage.filters
import numba
import gc

from multiprocessing import Pool, Process

%load_ext memory_profiler
%load_ext line_profiler
import timeit

from IPython.display import HTML

%matplotlib inline

## 2D Discrete Laplacian

In continuous form : 
$$ U_t = \triangle U - \lambda U $$

In discrete form, for point $i$ : 
$$ \Delta U_i = \sum_{1 = w(i,j)}\omega(i,j)(U_i - U_j) - \lambda U_i $$

Use discrete laplacian approximation w/o diagonals for grid spacing, so that we can have zero-flux  boundary conditions. 

$$ L = 
 \begin{pmatrix}
  0 & 1 & 0 \\
  1 & -4 & 1 \\
  0 & 1 & 0 
 \end{pmatrix} $$

I use a convolution function to calculate the diffusion terms. 

# Helper functions used to define the arenas 
### Needs
* read excel or csv files 
* rescaling arrays and contents 
* convert row/col to array index


* disk function, projects circular areas onto an input grid 
* 

In [11]:
import 

In [12]:
scale = np.power(2,4).astype(np.int)
tmax=60

# Params :    ,Dc,    rc,  rp,   Kn,   Dn,  kn,  Da,  xa,    xs,  xS,
p0 = np.array([0,    8e-3,  8e-3, 15,  3e-2,  1, 3e-2, 8e-2, 5e2, 2e2, 
#xr,   hS,   kS,  hR, kR,  hC,  kC,    pa,  colthresh   od0
1.2e3, 3.2, 100, 2.3, 55, 3.2, 2.7e3, 5e-3, 1e-3, 0.1], dtype=np.float32)
# Define parameter values to sample
Dc,rc,rp,Kn,Dn,kn,Da,xa,xs,xS,xr,hS,kS,hR,kR,hC,kC,pa,col_thresh,od0 = p0


# print('scale:{} solver:{} pad0'.format(scale, solver))
out = sim_pad(p0, scale, tmax, prep_pad_0)

NameError: name 'c_i' is not defined

In [ ]:
# scale = 4
# p0 = np.array([1e-5, 3e-1, 2e-2, 75, 2, 1, 1e-8, 8e-3, 5e2, 1], dtype=np.float32)
# Dc, Dn,    rc, Kn, Hn, pn, xs,   ps, leak, od0 = p0
# dx = np.power((scale/4.5),2)
# p1 = np.array([dx, Dc, Dn, rc, Kn, Hn, pn, xs, ps, leak, od0])
# species, n_h, n_w, scale = dims
# # plt.imshow(calc_jac(0,initial_array.flatten())==0)
# # plt.imshow(calc_jac(0,initial_array.flatten())==0)
# n_vars = n_w*n_h*species
# initial_vec = initial_array.flatten()
# args=(np.zeros((species, n_h, n_w), dtype=np.float32,order='C'), 
#       np.zeros((species, n_h, n_w), dtype=np.float32,order='C'), 
#       np.zeros((n_h, n_w), dtype=np.float32,order='C'), 
#       p1)
# d_y, diff_terms, nut_avail, _ = args
# calc_lam = lambda y : calc_f(y,*args)
# %timeit calc_jac(0,initial_vec)
# # %timeit calc_diff_jac()
# # %timeit calc_jac(0,initial_array.flatten())
# # plt.imshow(jac_arr==0)
# # plt.figure(figsize=(20,20))
# # calc_arr = calc_jac(0,initial_array.flatten())
# # plt.imshow(calc_arr==0)
# # plt.figure(figsize=(20,20))
# # plt.imshow(np.isclose(jac_arr, calc_arr, 0.001))

In [ ]:
plt.close('all')
scale = np.power(2,6).astype(np.int)
tmax = 10*60

# Params :    ,           dx               Dc,    rc,  rp,   Kn,   Dn,  kn,  Da,  xa,    xs,  xS,
p0 = np.array([np.power((scale/4.5),2),   0,    8e-3,  8e-3, 15,  3e-2,  1, 3e-2, 8e-2, 5e2, 2e2, 
#xr,   hS,   kS,  hR, kR,  hC,  kC,    pa,  leak   od0
1.2e3, 3.2, 100, 2.3, 55, 3.2, 2.7e3, 5e-3, 1e-8, 0.1], dtype=np.float32)
# Define parameter values to sample
Dc,rc,rp,Kn,Dn,kn,Da,xa,xs,xS,xr,hS,kS,hR,kR,hC,kC,pa,leak,od0 = p0
rc_vec = [rc]
n_rc = len(rc_vec)
dn_vec = [Dn]
n_dn = len(dn_vec)
kn_vec = [Kn]
n_kn = len(kn_vec)
pn_vec = np.linspace(1,20,4)
n_pn = len(pn_vec)
od0_vec = [0.1,1,10]
n_od0 = len(od0_vec)

# Create parameter data frame
columns = 'Dc' ,'Dn' ,'rc' ,'Kn' ,'Hn' ,'pn' ,'xs' ,'ps' ,'leak' ,'od0'
index = np.arange(n_rc*n_dn*n_kn*n_pn*n_od0)
p0_df = pd.DataFrame(columns=columns, index=index)

i = 0
for rc in rc_vec:
    for Dn in dn_vec:
        for Kn in kn_vec:
            for pn in pn_vec:
                for od0 in od0_vec:
                    p0_df.iloc[i,:] = Dc, Dn,    rc, Kn, Hn, pn, xs,   ps, leak, od0
                    i += 1


# for scale_ind in np.arange(5,8):
#     for solver in ['RK45','LSODA']:
#         scale = np.power(2,scale_ind).astype(np.int)
#         print('scale:{} solver:{} pad0'.format(scale, solver))
#         %time sim_pad(p0_df.loc[0,:].values, scale, tmax, prep_pad_0)

# pad_fn_list = [prep_pad_0,prep_pad_1,prep_pad_2,prep_pad_3,prep_pad_4,prep_pad_5,prep_pad_6,prep_pad_7,prep_pad_8]
# for scale_ind in [3]:
#     for i, pad_fn in enumerate(pad_fn_list):
#         scale = np.power(2,scale_ind).astype(np.int)
#         print(scale, i)
#         %time sim_pad(p0_df.iloc[1,:].values, scale, tmax, pad_fn)
    
def worker(p0_df):
    pad_fns = [prep_pad_0,prep_pad_1,prep_pad_2,prep_pad_3,prep_pad_4,prep_pad_5,prep_pad_6,prep_pad_7,prep_pad_8]
    for p0_ind in p0_df.index:
        p0 = p0_df.loc[p0_ind,:].values
        for pad_ind, pad_fn in enumerate(pad_fns):
            out = sim_pad(p0, scale, tmax, pad_fn)
            np.save('worker_outputs/pad{}_p{}_arr.npy'.format(pad_ind, p0_ind), out[0])
            np.save('worker_outputs/pad{}_p{}_tvc.npy'.format(pad_ind, p0_ind), out[1])
#             anim = write_movie(out[0], out[1])
#             anim.save('worker_outputs//animation_pad{}_p{}.gif'.format(pad_ind,p0_ind), writer='pillow')    
    
jobs = []
n_proc = 8
for i in range(n_proc):
    p = Process(target=worker, args=(p0_df[i::n_proc].copy(),))
    jobs.append(p)
    p.start()
p.join()

# plt.figure(figsize=(20,20))
# plt.imshow(prep_pad_4(scale))


In [ ]:
c_i, n_i, s_i = np.arange(3)
def write_movie(im_arr, t_vec, skip=1, n_frames=200):
    
    frames, s, h, w = im_arr.shape
    t_points = np.arange(0,t_vec.max(),n_frames)
    f_points = np.arange(frames)
    
    #frames = len(t)
    
    t, s, h, w = im_arr.shape
    xticks = []
    xticklabels = []
    # First set up the figure, the axis, and the plot element we want to animate
    blank_array = np.zeros([h, w])
    fig, axs = plt.subplots(1,3, figsize=(10,7))
    im_list = [0,0,0]
    
    # Plot cell densities
    ax = axs[0]
    indxs = [c_i]
    vmax = im_arr[-1,indxs,:,:].sum(axis=0).max()
    vmin = im_arr[-1,indxs,:,:].sum(axis=0).min()
    im = ax.imshow(blank_array, animated=True, vmax=vmax, vmin=vmin, interpolation='none', aspect=1)
    cbar = fig.colorbar(im, ax=ax, ticks=[vmin, vmax])
    ax.set_xticks([])
    ax.set_xticklabels(xticklabels)
    ax.set_yticks([])
    ax.set_title('cell densities')
    im_list[0] = im
    
    # Plot nutrient densities
    ax = axs[1]
    indxs = [n_i]
    vmax = im_arr[:,indxs[0],:,:].max()
    vmin = im_arr[:,indxs[0],:,:].min()
    im = ax.imshow(blank_array, animated=True, vmax=vmax, vmin=vmin, interpolation='none', aspect=1)
    cbar = fig.colorbar(im, ax=ax, ticks=[vmin, vmax])
    ax.set_xticks(xticks)
    ax.set_xticklabels(xticklabels)
    ax.set_yticks([])
    ax.set_title('nutrient')
    im_list[1] = im
    
    # Plot synthase densities
    ax = axs[2]
    indxs = [s_i]
    v_arr = im_arr[:,indxs,:,w//2:].prod(axis=1)
    vmax = v_arr.max()
    vmin = v_arr.min()
    im = ax.imshow(blank_array, animated=True, vmax=vmax, vmin=vmin, interpolation='none', aspect=1)
    cbar = fig.colorbar(im, ax=ax, ticks=[vmin, vmax])
    ax.set_xticks(xticks)
    ax.set_xticklabels(xticklabels)
    ax.set_yticks([])
    ax.set_title('synthases')
    im_list[2] = im

    # animation function.  This is called sequentially
    t_points = np.linspace(0,t_vec.max(), 200)
    f_inds = []
    t_ind = 0
    for tp in t_points:
        while tp > t_vec[t_ind]:
            t_ind += 1
        f_inds.append(t_ind-1)
        
    def animate(t_point):
        i = f_inds[t_point]
        
        # Plot cell densities
        ax = axs[0]
        indxs = [c_i]
        frame_arr = im_arr[i,indxs,:,:].sum(axis=0)
        im_list[0].set_array(frame_arr)

        # Plot nutrient densities
        ax = axs[1]
        indxs = [n_i]
        frame_arr = im_arr[i,indxs,:,:].sum(axis=0)
        im_list[1].set_array(frame_arr)

        # Plot synthase densities
        ax = axs[2]
        indxs = [s_i]
        frame_arr = im_arr[i,indxs,:,:].prod(axis=0)
        im_list[2].set_array(frame_arr)

        #return im_list,

    # call the animator.  blit=True means only re-draw the parts that have changed.
    anim = anm.FuncAnimation(fig, animate, interval=50, frames=n_frames)

#     anim.save('./animation_test.gif', writer='pillow')
    fig.tight_layout()
    plt.close('all')
    return anim
#     HTML(anim.to_html5_video())


In [ ]:
plt.close('all')
pad_size = []
gb_p = sim_df.groupby('p_ind')
for p0_ind in p0_df.index:
    plt.figure()
    sub_df = gb_p.get_group(p0_ind)
    for pad_ind in sub_df.pad_ind.values:
        tvec_fn, avec_fn = p0_df.loc[p0_ind, ['pad{}_avec_fn'.format(pad_ind), 'pad{}_tvec_fn'.format(pad_ind)]]
        if 'na' in [tvec_fn, avec_fn]:
            pass
        else:
            img = np.load('worker_outputs/pad{}_p{}_arr.npy'.format(pad_ind, p0_ind))
            t_vec = np.load('worker_outputs/pad{}_p{}_tvc.npy'.format(pad_ind, p0_ind))
        #     plt.figure()
        #     dy_vec = (np.diff(img[:,c_i,:,:],axis=0)/img[1:,c_i,:,:]).flatten('F')
            c_vec = img[:,c_i,:,:].sum(axis=-1).sum(axis=-1)
            dy_vec = np.diff(c_vec)/np.diff(t_vec)/c_vec[:-1]
            tc_ind = np.arange(len(dy_vec))[dy_vec<0.8*dy_vec.max()]
            if len(tc_ind) == 0:
                tc_ind = len(dy_vec)-1
            else:
                tc_ind = tc_ind[0]
        #     dy_t_vec = np.tile(t_vec[1:], len(dy_vec)//len(t_vec[1:]))
            plt.plot(img.shape[2], t_vec[tc_ind]/60,'o',label=pad_ind)
            plt.title('Simulation threshold crossings {}'.format(p0_ind))
#     plt.legend()
    plt.tight_layout()
    plt.xlabel('Pad size')
    plt.ylabel('Threshold crossing (minutes)')
#     plt.ylim([-1,3])


In [ ]:
def gif_worker(sim_df):
    for ind in sim_df.index:
        p_ind, pad_ind = sim_df.loc[ind,['p_ind','pad_ind']]
        img = np.load('worker_outputs/pad{}_p{}_arr.npy'.format(pad_ind, p_ind))
        tvc = np.load('worker_outputs/pad{}_p{}_tvc.npy'.format(pad_ind, p_ind))
        anim = write_movie(img, tvc)
        anim.save('worker_outputs/animation_pad{}_p{}.gif'.format(pad_ind,p_ind), writer='pillow')    

jobs = []
n_proc = 12
for i in range(n_proc):
    p = Process(target=gif_worker, args=(sim_df[i::n_proc].copy(),))
    jobs.append(p)
    p.start()
# p.join()


In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(skimage.transform.downscale_local_mean(img[5,c_i,:,:],(2,2)))

In [ ]:
skimage.transform.downscale_local_mean?

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(out[0][-1,c_i,:,:])